# Traits and Generics

## Traits

* Traits are Rust’s take on interfaces or abstract base classes

In [55]:
trait Shape {
    fn move_by(&mut self, dx: i32, dy: i32);
    fn area(&self) -> f64;
}

### Implementing Traits

* To implement a trait for a type, we use the `impl` keyword

In [56]:
#[derive(Debug)]
struct Point {
    x: i32,
    y: i32
}

#[derive(Debug)]
struct Circle {
    position: Point,
    radius: u32
}

#[derive(Debug)]
struct Rectangle {
    position: Point,
    width: u32,
    height: u32
}

impl Shape for Circle {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        std::f64::consts::PI * (self.radius as f64) * (self.radius as f64)
    }
}

impl Shape for Rectangle {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        (self.width * self.height) as f64
    }
}

### Default Trait Behavior

* We can provide default implementation for traits

In [57]:
trait Drawable: std::fmt::Debug {
    fn draw(&self) {
        println!("Drawing a shape: {:?}", self);
    }
}

#[derive(Debug)]
struct Sprite {
    position: Point
}

impl Drawable for Sprite {
    // no need to implement draw - it will use the default implementation
}

impl Drawable for Circle {
    fn draw(&self) {
        println!("Drawing the circle at {:?} with radius: {}", self.position, self.radius);
    }
}

impl Drawable for Rectangle {
    fn draw(&self) {
          println!("Drawing the rectangle at {:?} with width: {} and height: {}", self.position, self.width, self.height);
  }
}

fn main() {
    let s = Sprite { position: Point { x: 0, y: 0 } };
    let c = Circle { position: Point { x: 0, y: 0 }, radius: 10 };
    let r = Rectangle { position: Point { x: 0, y: 0 }, width: 10, height: 20 };

    s.draw();
    c.draw();
    r.draw();
}

main();

### Trait Bounds

* Trait bounds specify that a generic can be any type that implements a trait. 
* They work like a contracts at compile time

#### `impl Trait` Syntax

* The `impl Trait` syntax is a shorthand for a longer form, called a trait bound

In [ ]:
fn draw_shape(shape: &impl Drawable) {
    shape.draw()
}

let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
draw_shape(&circle);

let rect = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};
draw_shape(&rect);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


Drawing a shape: Sprite { position: Point { x: 0, y: 0 } }


Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


#### Trait Bound Syntax

In [59]:
fn draw_shape<T: Drawable>(shape: &T) {
    shape.draw()
}

draw_shape(&circle);
draw_shape(&rect);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


#### Many trait bounds

* We can specify multiple trait bounds on a generic type by using the `+` syntax

In [60]:
fn use_shape(shape: &mut (impl Shape + Drawable)) {
    shape.move_by(10, 15);
    shape.draw();
}

In [61]:
fn use_shape<T: Shape + Drawable>(shape: &mut T) {
    shape.move_by(10, 15);
    shape.draw();
}

#### Clearer Trait Bounds with `where` Clauses

* We can use a `where` clause to specify trait bounds on generic types

In [62]:
fn some_function<T: Printable + Clone, U: Drawable + Clone>(t: &T, u: &U) {
    t.print();
    u.draw();
}

In [63]:
fn some_function<T, U>(t: &T, u: &U)
where 
    T: Printable + Clone,
    U: Drawable + Clone,
{
    t.print();
    u.draw();
}

### Returning Types that Implement Traits

In [64]:
fn create_drawable() -> Box<dyn Drawable> {
    Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5})
}

let shape: Box<dyn Drawable> = create_drawable();
shape.draw();

Drawing the circle at Point { x: 10, y: 20 } with radius: 5


### Self in Traits

* `Self` is an alias for the type we’re implementing the trait for

In [65]:
trait Cloneable {
    fn clone(&self) -> Self; // the type of x.clone() is the same as the type of x, whatever that might be
}

* A trait that uses the `Self` type is incompatible with trait objects:

In [66]:
fn do_clone(prototype: &dyn Cloneable) -> Box<dyn Cloneable> {
    let cloned_object = prototype.clone();
    Box::new(cloned_object)
}

Error: the trait `Cloneable` cannot be made into an object

Error: the trait `Cloneable` cannot be made into an object

Error: the size for values of type `dyn Cloneable` cannot be known at compilation time

Error: the trait `Cloneable` cannot be made into an object

Error: the size for values of type `dyn Cloneable` cannot be known at compilation time

Error: the trait `Cloneable` cannot be made into an object

Error: unused variable: `result`

### Trait Objects

* A trait object allows to implement dynamic polymorphism in Rust

In [67]:
let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};

let shape: dyn Shape = circle; // ERROR: Shape does not have a constant size

Error: mismatched types

Error: the size for values of type `dyn Shape` cannot be known at compilation time

Error: unused variable: `result`

#### References to Trait

* A reference to trait type, like `Shape` is called **trait object**. Like any other reference, a trait object points to some value, it has a lifetime, and it can be either mut or shared.
* In memory, a trait object is a **fat pointer** consisting of a pointer to the value, plus a pointer to a table representing that value’s type (virtual table).



In [68]:
fn main() {
    let mut shape: &dyn Shape;
    
    let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
    shape = &circle;
    println!("Shape area: {:.2}", shape.area());

    shape = &Rectangle{position: Point{x: 10, y: 20}, width: 10, height: 20};
    println!("Shape area: {:.2}", shape.area());
}

main();

Shape area: 78.54
Shape area: 200.00


#### Boxes to Trait

* We can also use a `Box` to store a trait object
* A `Box<dyn Trait>` is a trait object that points to a value on the heap

In [69]:
let box_circle: Box<Circle> = Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5});
let mut shape: Box<dyn Shape> = box_circle; // OK - Box<dyn Shape> is trait object
println!("Shape area: {:.2}", shape.area());

shape = Box::new(Rectangle{position: Point{x: 10, y: 20}, width: 10, height: 20});
println!("Shape area: {:.2}", shape.area());

Shape area: 78.54
Shape area: 200.00


## Dynamic Polymorphism in Rust

In [70]:
#[derive(Debug)]
struct Square {
    position: Point,
    side: u32
}   

impl Shape for Square {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        (self.side * self.side) as f64
    }
}

impl Drawable for Square {
    fn draw(&self) {
        println!("Drawing the square at {:?} with side: {}", self.position, self.side);
    }
}

fn main() {
    let mut shapes: Vec<Box<dyn Shape>> = vec![
        Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5}),
        Box::new(Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5}),
        Box::new(Square{position: Point{x: 70, y: 80}, side: 7})
    ];

    for shape in shapes.iter_mut() {
        shape.move_by(5, 5);
        println!("Shape area: {:.2}", shape.area());
    }
}

main();

Shape area: 78.54
Shape area: 50.00
Shape area: 49.00


### Combining Traits

In [71]:
trait DrawableShape: Shape + Drawable {} // Trait that combines Shape and Drawable

impl<T> DrawableShape for T where T: Shape + Drawable {} // Implement DrawableShape for any type that implements Shape and Drawable

fn use_shape(shape: &mut dyn DrawableShape) {
    shape.draw();
    shape.move_by(10, 15);
    shape.draw();
}

fn main() {
    let mut circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
    use_shape(&mut circle);

    let mut rect = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};
    use_shape(&mut rect);
}

main();

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the circle at Point { x: 20, y: 35 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5
Drawing the rectangle at Point { x: 50, y: 65 } with width: 10 and height: 5


## Associated Types

In [72]:
trait Iterator {
    type Item; // associated type
    fn next(&mut self) -> Option<Self::Item>;
}

struct Counter {
    value: u32,
    sentinel: u32
}

impl Counter {
    fn new(sentinel: u32) -> Counter {
        Counter{value: 0, sentinel}
    }
}

impl Iterator for Counter {
    type Item = u32;

    fn next(&mut self) -> Option<Self::Item> {
        if self.value < self.sentinel {
            let result = self.value;
            self.value += 1;
            Some(result)
        } else {
            None
        }
    }
}

fn collect_into_vector<I: Iterator>(mut iter: I) -> Vec<I::Item>
{
    let mut results = Vec::new();

    while let Some(value) = iter.next() {
        results.push(value);
    }

    results
}

fn main() {
    let mut counter = Counter::new(5);

    while let Some(value) = counter.next() {
        println!("Counter value: {}", value);
    }

    let numbers = collect_into_vector(Counter::new(5));
    println!("Collected numbers: {:?}", numbers);
}

main();

Counter value: 0
Counter value: 1
Counter value: 2
Counter value: 3
Counter value: 4
Collected numbers: [0, 1, 2, 3, 4]


### Subtraits

* A trait can be an extension of another trait:

In [73]:
trait BidirectionalIterator: Iterator {
    fn prev(&mut self) -> Option<Self::Item>;
}

impl BidirectionalIterator for Counter {
    fn prev(&mut self) -> Option<Self::Item> {
        if self.value > 0 {
            let result = self.value;
            self.value -= 1;
            Some(self.value)
        } else {
            None
        }
    }
}

fn main() {
    let mut counter = Counter::new(5);

    println!("Forward iteration:");
    while let Some(value) = counter.next() {
        println!("Counter value: {}", value);
    }

    println!("\nBackward iteration:");
    while let Some(value) = counter.prev() {
        println!("Counter value: {}", value);
    }
}

main();

Forward iteration:
Counter value: 0
Counter value: 1
Counter value: 2
Counter value: 3
Counter value: 4

Backward iteration:
Counter value: 4
Counter value: 3
Counter value: 2
Counter value: 1
Counter value: 0


## Generics

### Generic Function

In [74]:
fn min<T: Ord>(a: T, b: T) -> T {
    if a < b {
        a
    } else {
        b
    }
}

In [75]:
min(1, 2)

1

In [76]:
min("hello", "world")

"hello"

### Generic Structure

In [77]:
struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    fn push(&mut self, item: T) {
        self.items.push(item);
    }

    fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }
}

fn main() {
    let mut stack = Stack{items: Vec::new()};
    stack.push(1);
    stack.push(2);
    stack.push(3);

    while let Some(item) = stack.pop() {
        println!("Popped: {}", item);
    }
}

main();

Popped: 3
Popped: 2
Popped: 1


### Generic Traits

In [78]:
mod explain {
    /// std::ops::Mul, the trait for types that support `*`.
    pub trait Mul<RHS=Self> {
        /// The resulting type after applying the `*` operator
        type Output;

        /// The method for the `*` operator
        fn mul(self, rhs: RHS) -> Self::Output;
    }
}

In [79]:
#[derive(Debug)]
struct Complex {
    real: f64,
    imag: f64
}

use std::ops::Mul;

impl Mul for Complex {
    type Output = Complex;

    fn mul(self, rhs: Complex) -> Complex {
        Complex {
            real: self.real * rhs.real - self.imag * rhs.imag,
            imag: self.real * rhs.imag + self.imag * rhs.real
        }
    }
}

fn main() {
    let a = Complex{real: 1.0, imag: 2.0};
    let b = Complex{real: 3.0, imag: 4.0};

    let c = a.mul(b);
    println!("Result: {} + {}i", c.real, c.imag);
}

main();

Result: -5 + 10i


## Fully qualified methods

In [80]:
let mut circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
let mut rectangle = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};

circle.draw();
Shape::move_by(&mut circle, 10, 15);
Drawable::draw(&circle);

<Rectangle as Drawable>::draw(&rectangle);
rectangle.move_by(10, 15);
Drawable::draw(&rectangle);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the circle at Point { x: 20, y: 35 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5
Drawing the rectangle at Point { x: 50, y: 65 } with width: 10 and height: 5


* Fully qualified method calls are used:
  * to disambiguate between methods with the same name